## 1. Setup and Imports

In [27]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ks_2samp, mannwhitneyu
import warnings
import os
from io import StringIO

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✓ Libraries imported successfully")
print("✓ Configuration set")

✓ Libraries imported successfully
✓ Configuration set


## 2. Load Data

In [28]:
# Define file paths
input_filepath = 'Raw_Data/credit_card_fraud_10k.csv'
output_filepath = 'Cleaned_Data/credit_card_fraud_10k_cleaned.csv'

# Create output directory if it doesn't exist
output_dir = os.path.dirname(output_filepath)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created output directory: {output_dir}")

# Load data
print(f"Loading data from: {input_filepath}")
df_raw = pd.read_csv(input_filepath)
print(f"✓ Loaded {len(df_raw)} records from file")
print("="*80)
print("FRAUD DETECTION DATA PIPELINE - INITIALIZED")
print("="*80)

# Create a copy for cleaning
df = df_raw.copy()
print(f"\n✓ Created working copy of data")

Loading data from: Raw_Data/credit_card_fraud_10k.csv
✓ Loaded 10000 records from file
FRAUD DETECTION DATA PIPELINE - INITIALIZED

✓ Created working copy of data


## 3. Data Quality Assessment (BEFORE Cleaning)
### Step 1: Schema Validation

In [29]:
print("\n[STEP 1] SCHEMA VALIDATION")
print("-" * 80)

expected_schema = {
    'transaction_id': 'int64',
    'amount': 'float64',
    'transaction_hour': 'int64',
    'merchant_category': 'object',
    'foreign_transaction': 'int64',
    'location_mismatch': 'int64',
    'device_trust_score': 'int64',
    'velocity_last_24h': 'int64',
    'cardholder_age': 'int64',
    'is_fraud': 'int64'
}

schema_valid = True
for col, dtype in expected_schema.items():
    if col not in df.columns:
        print(f"  ✗ Missing column: {col}")
        schema_valid = False
    elif df[col].dtype != dtype:
        print(f"  ⚠ Column {col}: Expected {dtype}, got {df[col].dtype}")

if schema_valid:
    print("  ✓ Schema validation PASSED")


[STEP 1] SCHEMA VALIDATION
--------------------------------------------------------------------------------
  ✓ Schema validation PASSED


### Step 2: Missing Value Analysis

In [30]:
print("\n[STEP 2] MISSING VALUE ANALYSIS")
print("-" * 80)

missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).values,
    'Data_Type': df.dtypes.values
})

print(missing_summary.to_string(index=False))

total_missing = df.isnull().sum().sum()
if total_missing == 0:
    print("\n  ✓ No missing values detected")
else:
    print(f"\n  ⚠ Total missing values: {total_missing}")


[STEP 2] MISSING VALUE ANALYSIS
--------------------------------------------------------------------------------
             Column  Missing_Count  Missing_Percentage Data_Type
     transaction_id              0                 0.0     int64
             amount              0                 0.0   float64
   transaction_hour              0                 0.0     int64
  merchant_category              0                 0.0    object
foreign_transaction              0                 0.0     int64
  location_mismatch              0                 0.0     int64
 device_trust_score              0                 0.0     int64
  velocity_last_24h              0                 0.0     int64
     cardholder_age              0                 0.0     int64
           is_fraud              0                 0.0     int64

  ✓ No missing values detected


### Step 3: Duplicate Detection

In [31]:
print("\n[STEP 3] DUPLICATE DETECTION")
print("-" * 80)

# Check duplicate rows
duplicate_rows = df.duplicated()
dup_count = duplicate_rows.sum()

# Check duplicate transaction IDs
dup_ids = df['transaction_id'].duplicated()
dup_id_count = dup_ids.sum()

print(f"  Duplicate Rows: {dup_count} ({dup_count/len(df)*100:.2f}%)")
print(f"  Duplicate Transaction IDs: {dup_id_count}")

if dup_count > 0:
    print("\n  Duplicate Records:")
    print(df[duplicate_rows])
else:
    print("\n  ✓ No duplicates detected")


[STEP 3] DUPLICATE DETECTION
--------------------------------------------------------------------------------
  Duplicate Rows: 0 (0.00%)
  Duplicate Transaction IDs: 0

  ✓ No duplicates detected


### Step 4: Domain Constraint Validation

In [32]:
print("\n[STEP 4] DOMAIN CONSTRAINT VALIDATION (BEFORE CLEANING)")
print("-" * 80)

validation_results = {}

# Transaction Hour: 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
validation_results['transaction_hour'] = len(invalid_hours)
print(f"  transaction_hour (0-23): {len(invalid_hours)} invalid values")
if len(invalid_hours) > 0:
    print(f"    → Invalid values: {invalid_hours['transaction_hour'].unique()}")

# Device Trust Score: 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
validation_results['device_trust_score'] = len(invalid_scores)
print(f"  device_trust_score (0-100): {len(invalid_scores)} invalid values")
if len(invalid_scores) > 0:
    print(f"    → Invalid values: {invalid_scores['device_trust_score'].unique()}")

# Amount: positive
invalid_amounts = df[df['amount'] <= 0]
validation_results['amount'] = len(invalid_amounts)
print(f"  amount (>0): {len(invalid_amounts)} invalid values")
if len(invalid_amounts) > 0:
    print(f"    → Invalid transaction IDs: {invalid_amounts['transaction_id'].tolist()}")
    print(f"    → Invalid amounts: {invalid_amounts['amount'].tolist()}")

# Velocity: non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
validation_results['velocity'] = len(invalid_velocity)
print(f"  velocity_last_24h (≥0): {len(invalid_velocity)} invalid values")
if len(invalid_velocity) > 0:
    print(f"    → Invalid values: {invalid_velocity['velocity_last_24h'].unique()}")

# Age: reasonable range
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
validation_results['cardholder_age'] = len(invalid_age)
print(f"  cardholder_age (18-100): {len(invalid_age)} invalid values")
if len(invalid_age) > 0:
    print(f"    → Invalid values: {invalid_age['cardholder_age'].unique()}")

# Binary fields: 0 or 1
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    validation_results[field] = len(invalid_binary)
    print(f"  {field} (0/1): {len(invalid_binary)} invalid values")
    if len(invalid_binary) > 0:
        print(f"    → Invalid values: {invalid_binary[field].unique()}")

total_invalid = sum(validation_results.values())
if total_invalid == 0:
    print("\n  ✓ All domain constraints satisfied")
else:
    print(f"\n  ⚠ Total domain violations: {total_invalid}")
    print(f"  → These will be CLEANED in the next steps")


[STEP 4] DOMAIN CONSTRAINT VALIDATION (BEFORE CLEANING)
--------------------------------------------------------------------------------
  transaction_hour (0-23): 0 invalid values
  device_trust_score (0-100): 0 invalid values
  amount (>0): 1 invalid values
    → Invalid transaction IDs: [2903]
    → Invalid amounts: [0.0]
  velocity_last_24h (≥0): 0 invalid values
  cardholder_age (18-100): 0 invalid values
  foreign_transaction (0/1): 0 invalid values
  location_mismatch (0/1): 0 invalid values
  is_fraud (0/1): 0 invalid values

  ⚠ Total domain violations: 1
  → These will be CLEANED in the next steps


### Step 5: Outlier Detection

In [33]:
print("\n[STEP 5] OUTLIER DETECTION (BEFORE CLEANING)")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

outlier_summary = []

for feature in numerical_features:
    # IQR Method
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    
    # Z-Score Method (threshold = 3)
    z_scores = np.abs(stats.zscore(df[feature]))
    z_outliers = df[z_scores > 3]
    
    outlier_summary.append({
        'Feature': feature,
        'IQR_Outliers': len(iqr_outliers),
        'Z_Score_Outliers': len(z_outliers),
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound,
        'Min': df[feature].min(),
        'Max': df[feature].max()
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))
print("\n  Note: Outliers in fraud detection are often IMPORTANT (fraud cases)")
print("        We will keep them unless they are clearly data errors")


[STEP 5] OUTLIER DETECTION (BEFORE CLEANING)
--------------------------------------------------------------------------------
           Feature  IQR_Outliers  Z_Score_Outliers  Lower_Bound  Upper_Bound  Min     Max
            amount           501               180    -236.4575     529.8425  0.0 1471.04
  transaction_hour             0                 0     -12.0000      36.0000  0.0   23.00
device_trust_score             0                 0     -12.5000     135.5000 25.0   99.00
 velocity_last_24h            51                51      -2.0000       6.0000  0.0    9.00
    cardholder_age             0                 0      -9.0000      95.0000 18.0   69.00

  Note: Outliers in fraud detection are often IMPORTANT (fraud cases)
        We will keep them unless they are clearly data errors


## 4. 🧹 DATA CLEANING (ทำความสะอาดจริงๆ)
### เริ่มต้นการ Clean ข้อมูล

In [34]:
print("\n" + "="*80)
print("🧹 DATA CLEANING PROCESS (การทำความสะอาดข้อมูลจริง)")
print("="*80)

rows_before = len(df)
cleaning_log = []

print(f"\nStarting with: {rows_before} records")
print("\nCleaning steps:")


🧹 DATA CLEANING PROCESS (การทำความสะอาดข้อมูลจริง)

Starting with: 10000 records

Cleaning steps:


### Clean 1: Remove/Fix Invalid Amounts

In [35]:
print("\n[CLEAN 1] Handling Invalid Amounts (amount <= 0)")
print("-" * 80)

invalid_amounts = df[df['amount'] <= 0]
print(f"  Found {len(invalid_amounts)} records with invalid amounts")

if len(invalid_amounts) > 0:
    print(f"\n  Invalid records:")
    print(invalid_amounts[['transaction_id', 'amount', 'is_fraud']].to_string(index=False))
    
    # Decision: Remove these records (can't have transactions with zero/negative amounts)
    df = df[df['amount'] > 0]
    
    removed = len(invalid_amounts)
    print(f"\n  ✓ Removed {removed} records with invalid amounts")
    cleaning_log.append(f"Removed {removed} records: amount <= 0")
else:
    print("  ✓ No invalid amounts found")

print(f"\n  Records after cleaning: {len(df)} (removed: {rows_before - len(df)})")


[CLEAN 1] Handling Invalid Amounts (amount <= 0)
--------------------------------------------------------------------------------
  Found 1 records with invalid amounts

  Invalid records:
 transaction_id  amount  is_fraud
           2903     0.0         0

  ✓ Removed 1 records with invalid amounts

  Records after cleaning: 9999 (removed: 1)


### Clean 2: Remove Duplicates (if any)

In [36]:
print("\n[CLEAN 2] Removing Duplicate Records")
print("-" * 80)

rows_before_dedup = len(df)
df_before_dedup = df.copy()

# Remove duplicate rows (keep first occurrence)
df = df.drop_duplicates()

duplicates_removed = rows_before_dedup - len(df)
if duplicates_removed > 0:
    print(f"  ✓ Removed {duplicates_removed} duplicate records")
    cleaning_log.append(f"Removed {duplicates_removed} duplicate records")
else:
    print("  ✓ No duplicates found")

# Check for duplicate transaction IDs
dup_ids = df['transaction_id'].duplicated()
if dup_ids.sum() > 0:
    print(f"\n  ⚠ Warning: {dup_ids.sum()} duplicate transaction IDs still exist")
    print("  → Removing records with duplicate transaction IDs (keeping first)")
    df = df.drop_duplicates(subset=['transaction_id'], keep='first')
    print(f"  ✓ Cleaned duplicate IDs")
    cleaning_log.append(f"Removed duplicate transaction IDs")

print(f"\n  Records after deduplication: {len(df)}")


[CLEAN 2] Removing Duplicate Records
--------------------------------------------------------------------------------
  ✓ No duplicates found

  Records after deduplication: 9999


### Clean 3: Fix Domain Violations

In [37]:
print("\n[CLEAN 3] Fixing Domain Constraint Violations")
print("-" * 80)

rows_before_domain = len(df)

# Transaction Hour: clip to 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
if len(invalid_hours) > 0:
    print(f"  ⚠ Found {len(invalid_hours)} invalid transaction hours")
    df['transaction_hour'] = df['transaction_hour'].clip(0, 23)
    print(f"  ✓ Clipped transaction_hour to valid range [0-23]")
    cleaning_log.append(f"Clipped {len(invalid_hours)} transaction_hour values")

# Device Trust Score: clip to 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
if len(invalid_scores) > 0:
    print(f"  ⚠ Found {len(invalid_scores)} invalid device trust scores")
    df['device_trust_score'] = df['device_trust_score'].clip(0, 100)
    print(f"  ✓ Clipped device_trust_score to valid range [0-100]")
    cleaning_log.append(f"Clipped {len(invalid_scores)} device_trust_score values")

# Velocity: clip to non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
if len(invalid_velocity) > 0:
    print(f"  ⚠ Found {len(invalid_velocity)} negative velocity values")
    df['velocity_last_24h'] = df['velocity_last_24h'].clip(lower=0)
    print(f"  ✓ Clipped velocity_last_24h to non-negative")
    cleaning_log.append(f"Clipped {len(invalid_velocity)} velocity_last_24h values")

# Age: Remove records with invalid ages (can't fix age easily)
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
if len(invalid_age) > 0:
    print(f"  ⚠ Found {len(invalid_age)} invalid ages")
    df = df[(df['cardholder_age'] >= 18) & (df['cardholder_age'] <= 100)]
    print(f"  ✓ Removed {len(invalid_age)} records with invalid ages")
    cleaning_log.append(f"Removed {len(invalid_age)} records: invalid age")

# Binary fields: Remove records with invalid binary values
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    if len(invalid_binary) > 0:
        print(f"  ⚠ Found {len(invalid_binary)} invalid {field} values")
        df = df[df[field].isin([0, 1])]
        print(f"  ✓ Removed {len(invalid_binary)} records with invalid {field}")
        cleaning_log.append(f"Removed {len(invalid_binary)} records: invalid {field}")

domain_removed = rows_before_domain - len(df)
if domain_removed == 0:
    print("\n  ✓ No domain violations needed cleaning")
else:
    print(f"\n  Total records removed for domain violations: {domain_removed}")

print(f"\n  Records after domain cleaning: {len(df)}")


[CLEAN 3] Fixing Domain Constraint Violations
--------------------------------------------------------------------------------

  ✓ No domain violations needed cleaning

  Records after domain cleaning: 9999


### Clean 4: Handle Missing Values (if any found)

In [38]:
print("\n[CLEAN 4] Handling Missing Values")
print("-" * 80)

missing_before = df.isnull().sum().sum()

if missing_before > 0:
    print(f"  Found {missing_before} missing values")
    print("\n  Missing value distribution:")
    print(df.isnull().sum()[df.isnull().sum() > 0])
    
    # Strategy: Remove rows with ANY missing values (for fraud detection, we need complete data)
    rows_with_missing = df.isnull().any(axis=1).sum()
    df = df.dropna()
    
    print(f"\n  ✓ Removed {rows_with_missing} rows with missing values")
    cleaning_log.append(f"Removed {rows_with_missing} rows with missing values")
else:
    print("  ✓ No missing values found")

print(f"\n  Records after missing value handling: {len(df)}")


[CLEAN 4] Handling Missing Values
--------------------------------------------------------------------------------
  ✓ No missing values found

  Records after missing value handling: 9999


### Clean 5: Reset Index

In [39]:
print("\n[CLEAN 5] Resetting Index")
print("-" * 80)

df = df.reset_index(drop=True)
print("  ✓ Index reset to sequential order")
print(f"\n  Final cleaned dataset: {len(df)} records")


[CLEAN 5] Resetting Index
--------------------------------------------------------------------------------
  ✓ Index reset to sequential order

  Final cleaned dataset: 9999 records


### Cleaning Summary

In [40]:
print("\n" + "="*80)
print("🧹 DATA CLEANING SUMMARY")
print("="*80)

print(f"\nOriginal dataset:    {rows_before:,} records")
print(f"Cleaned dataset:     {len(df):,} records")
print(f"Records removed:     {rows_before - len(df):,} ({(rows_before - len(df))/rows_before*100:.2f}%)")
print(f"Records retained:    {len(df)/rows_before*100:.2f}%")

print("\nCleaning actions performed:")
if cleaning_log:
    for i, action in enumerate(cleaning_log, 1):
        print(f"  {i}. {action}")
else:
    print("  ✓ No cleaning needed - data was already clean")

print("\n" + "="*80)


🧹 DATA CLEANING SUMMARY

Original dataset:    10,000 records
Cleaned dataset:     9,999 records
Records removed:     1 (0.01%)
Records retained:    99.99%

Cleaning actions performed:
  1. Removed 1 records: amount <= 0



## 5. Data Quality Assessment (AFTER Cleaning)
### Verify Domain Constraints Again

In [41]:
print("\n[VERIFICATION] DOMAIN CONSTRAINT VALIDATION (AFTER CLEANING)")
print("-" * 80)

verification_results = {}

# Transaction Hour: 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
verification_results['transaction_hour'] = len(invalid_hours)
print(f"  transaction_hour (0-23): {len(invalid_hours)} invalid values {'✓' if len(invalid_hours)==0 else '✗'}")

# Device Trust Score: 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
verification_results['device_trust_score'] = len(invalid_scores)
print(f"  device_trust_score (0-100): {len(invalid_scores)} invalid values {'✓' if len(invalid_scores)==0 else '✗'}")

# Amount: positive
invalid_amounts = df[df['amount'] <= 0]
verification_results['amount'] = len(invalid_amounts)
print(f"  amount (>0): {len(invalid_amounts)} invalid values {'✓' if len(invalid_amounts)==0 else '✗'}")

# Velocity: non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
verification_results['velocity'] = len(invalid_velocity)
print(f"  velocity_last_24h (≥0): {len(invalid_velocity)} invalid values {'✓' if len(invalid_velocity)==0 else '✗'}")

# Age: reasonable range
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
verification_results['cardholder_age'] = len(invalid_age)
print(f"  cardholder_age (18-100): {len(invalid_age)} invalid values {'✓' if len(invalid_age)==0 else '✗'}")

# Binary fields: 0 or 1
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    verification_results[field] = len(invalid_binary)
    print(f"  {field} (0/1): {len(invalid_binary)} invalid values {'✓' if len(invalid_binary)==0 else '✗'}")

total_invalid = sum(verification_results.values())
if total_invalid == 0:
    print("\n  ✅ All domain constraints satisfied - DATA IS CLEAN!")
else:
    print(f"\n  ❌ Total domain violations: {total_invalid} - CLEANING FAILED!")


[VERIFICATION] DOMAIN CONSTRAINT VALIDATION (AFTER CLEANING)
--------------------------------------------------------------------------------
  transaction_hour (0-23): 0 invalid values ✓
  device_trust_score (0-100): 0 invalid values ✓
  amount (>0): 0 invalid values ✓
  velocity_last_24h (≥0): 0 invalid values ✓
  cardholder_age (18-100): 0 invalid values ✓
  foreign_transaction (0/1): 0 invalid values ✓
  location_mismatch (0/1): 0 invalid values ✓
  is_fraud (0/1): 0 invalid values ✓

  ✅ All domain constraints satisfied - DATA IS CLEAN!


### Statistical Profiling (Cleaned Data)

In [42]:
print("\n[STEP 6] STATISTICAL PROFILING (CLEANED DATA)")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

stats_profile = df[numerical_features].describe(
    percentiles=[.01, .05, .25, .5, .75, .95, .99]
).T

# Add additional statistics
stats_profile['skewness'] = df[numerical_features].skew()
stats_profile['kurtosis'] = df[numerical_features].kurtosis()
stats_profile['cv'] = stats_profile['std'] / stats_profile['mean']

print("\nNumerical Features Profile:")
print(stats_profile.round(3))


[STEP 6] STATISTICAL PROFILING (CLEANED DATA)
--------------------------------------------------------------------------------

Numerical Features Profile:
                     count     mean      std    min     1%      5%     25%    50%     75%      95%      99%      max  skewness  kurtosis     cv
amount              9999.0  175.967  175.393   0.01   1.96   9.339  50.915  122.1  242.55  530.217  796.327  1471.04     1.919     5.118  0.997
transaction_hour    9999.0   11.593    6.923   0.00   0.00   1.000   6.000   12.0   18.00   22.000   23.000    23.00    -0.026    -1.206  0.597
device_trust_score  9999.0   61.798   21.488  25.00  25.00  28.000  43.000   62.0   80.00   96.000   99.000    99.00     0.011    -1.180  0.348
velocity_last_24h   9999.0    2.009    1.433   0.00   0.00   0.000   1.000    2.0    3.00    5.000    6.000     9.00     0.708     0.445  0.713
cardholder_age      9999.0   43.471   14.978  18.00  18.00  20.000  30.000   44.0   56.00   67.000   69.000    69.00     0.

### Target Variable Analysis (Cleaned Data)

In [43]:
print("\n[STEP 7] TARGET VARIABLE ANALYSIS (CLEANED DATA)")
print("-" * 80)

fraud_dist = df['is_fraud'].value_counts().sort_index()
fraud_pct = df['is_fraud'].value_counts(normalize=True).sort_index() * 100

print("\nTarget Variable Distribution:")
print(f"  Normal (0): {fraud_dist.get(0, 0)} ({fraud_pct.get(0, 0):.2f}%)")
print(f"  Fraud (1):  {fraud_dist.get(1, 0)} ({fraud_pct.get(1, 0):.2f}%)")

if 1 in fraud_dist.index and 0 in fraud_dist.index:
    imbalance_ratio = fraud_dist[0] / fraud_dist[1]
    print(f"\n  Imbalance Ratio: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 10:
        print("  ⚠ HIGH IMBALANCE - Consider resampling techniques")
    elif imbalance_ratio > 5:
        print("  ⚠ MODERATE IMBALANCE - Monitor model performance")
    else:
        print("  ✓ ACCEPTABLE BALANCE")


[STEP 7] TARGET VARIABLE ANALYSIS (CLEANED DATA)
--------------------------------------------------------------------------------

Target Variable Distribution:
  Normal (0): 9848 (98.49%)
  Fraud (1):  151 (1.51%)

  Imbalance Ratio: 65.22:1
  ⚠ HIGH IMBALANCE - Consider resampling techniques


### Correlation Analysis (Cleaned Data)

In [44]:
print("\n[STEP 9] CORRELATION ANALYSIS (CLEANED DATA)")
print("-" * 80)

numerical_cols = ['amount', 'transaction_hour', 'device_trust_score', 
                 'velocity_last_24h', 'cardholder_age', 
                 'foreign_transaction', 'location_mismatch', 'is_fraud']

corr_matrix = df[numerical_cols].corr()

print("\nCorrelation with Target Variable (is_fraud):")
target_corr = corr_matrix['is_fraud'].sort_values(ascending=False)
for feature, corr_val in target_corr.items():
    if feature != 'is_fraud':
        print(f"  {feature:25s}: {corr_val:7.4f}")

# High correlations between features
print("\nHigh Feature Correlations (|r| > 0.7):")
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    for feat1, feat2, corr_val in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: {corr_val:.4f}")
else:
    print("  ✓ No multicollinearity detected")


[STEP 9] CORRELATION ANALYSIS (CLEANED DATA)
--------------------------------------------------------------------------------

Correlation with Target Variable (is_fraud):
  foreign_transaction      :  0.1856
  location_mismatch        :  0.1730
  velocity_last_24h        :  0.1034
  amount                   :  0.0284
  cardholder_age           : -0.0006
  device_trust_score       : -0.1379
  transaction_hour         : -0.1387

High Feature Correlations (|r| > 0.7):
  ✓ No multicollinearity detected


## 6. Data Quality Report (Final)

In [45]:
print("\n" + "="*80)
print("📊 FINAL DATA QUALITY SCORECARD")
print("="*80)

total_records = len(df)
total_features = len(df.columns) - 1

# Calculate quality score
missing_values = df.isnull().sum().sum()
duplicates = df.duplicated().sum()
domain_violations = sum(verification_results.values())

quality_scores = {
    'Completeness': 100 if missing_values == 0 else 
                  (1 - missing_values / (total_records * total_features)) * 100,
    'Uniqueness': 100 if duplicates == 0 else
                 (1 - duplicates / total_records) * 100,
    'Validity': 100 if domain_violations == 0 else 0,
}

overall_quality = np.mean(list(quality_scores.values()))

print(f"\nDataset Size: {total_records} records × {total_features + 1} features")
print(f"\nQuality Dimensions:")
for dimension, score in quality_scores.items():
    status = "✓" if score >= 95 else "⚠"
    print(f"  {status} {dimension:15s}: {score:6.2f}%")

print(f"\n{'='*40}")
print(f"  Overall Quality Score: {overall_quality:.2f}%")
print(f"{'='*40}")

# Quality grade
if overall_quality >= 95:
    grade = "EXCELLENT ⭐⭐⭐"
elif overall_quality >= 85:
    grade = "GOOD ⭐⭐"
elif overall_quality >= 70:
    grade = "FAIR ⭐"
else:
    grade = "POOR ⚠"

print(f"\n  Data Quality Grade: {grade}")


📊 FINAL DATA QUALITY SCORECARD

Dataset Size: 9999 records × 10 features

Quality Dimensions:
  ✓ Completeness   : 100.00%
  ✓ Uniqueness     : 100.00%
  ✓ Validity       : 100.00%

  Overall Quality Score: 100.00%

  Data Quality Grade: EXCELLENT ⭐⭐⭐


## 7. Exploratory Data Analysis (Cleaned Data)
### Univariate Analysis

In [46]:
print("\n" + "="*80)
print("UNIVARIATE ANALYSIS (CLEANED DATA)")
print("="*80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

for feature in numerical_features:
    print(f"\n{feature.upper()}")
    print("-" * 40)
    
    print(f"  Mean:       {df[feature].mean():.2f}")
    print(f"  Median:     {df[feature].median():.2f}")
    print(f"  Std Dev:    {df[feature].std():.2f}")
    print(f"  Range:      [{df[feature].min():.2f}, {df[feature].max():.2f}]")
    print(f"  IQR:        {df[feature].quantile(0.75) - df[feature].quantile(0.25):.2f}")
    
    skew = df[feature].skew()
    kurt = df[feature].kurtosis()
    print(f"  Skewness:   {skew:.3f} {'(Right-skewed)' if skew > 0 else '(Left-skewed)' if skew < 0 else '(Symmetric)'}")
    print(f"  Kurtosis:   {kurt:.3f} {'(Heavy-tailed)' if kurt > 0 else '(Light-tailed)'}")


UNIVARIATE ANALYSIS (CLEANED DATA)

AMOUNT
----------------------------------------
  Mean:       175.97
  Median:     122.10
  Std Dev:    175.39
  Range:      [0.01, 1471.04]
  IQR:        191.64
  Skewness:   1.919 (Right-skewed)
  Kurtosis:   5.118 (Heavy-tailed)

TRANSACTION_HOUR
----------------------------------------
  Mean:       11.59
  Median:     12.00
  Std Dev:    6.92
  Range:      [0.00, 23.00]
  IQR:        12.00
  Skewness:   -0.026 (Left-skewed)
  Kurtosis:   -1.206 (Light-tailed)

DEVICE_TRUST_SCORE
----------------------------------------
  Mean:       61.80
  Median:     62.00
  Std Dev:    21.49
  Range:      [25.00, 99.00]
  IQR:        37.00
  Skewness:   0.011 (Right-skewed)
  Kurtosis:   -1.180 (Light-tailed)

VELOCITY_LAST_24H
----------------------------------------
  Mean:       2.01
  Median:     2.00
  Std Dev:    1.43
  Range:      [0.00, 9.00]
  IQR:        2.00
  Skewness:   0.708 (Right-skewed)
  Kurtosis:   0.445 (Heavy-tailed)

CARDHOLDER_AGE
----

### Bivariate Analysis

In [47]:
print("\n" + "="*80)
print("BIVARIATE ANALYSIS (Features vs Fraud) - CLEANED DATA")
print("="*80)

if df['is_fraud'].nunique() < 2:
    print("\n  ⚠ Cannot perform bivariate analysis - only one class present")
else:
    numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                         'velocity_last_24h', 'cardholder_age']
    
    for feature in numerical_features:
        print(f"\n{feature.upper()} by Fraud Status:")
        print("-" * 40)
        
        fraud_stats = df.groupby('is_fraud')[feature].describe()
        print(fraud_stats)
        
        normal_vals = df[df['is_fraud'] == 0][feature]
        fraud_vals = df[df['is_fraud'] == 1][feature]
        
        if len(fraud_vals) > 0:
            stat, p_value = mannwhitneyu(normal_vals, fraud_vals)
            print(f"\n  Mann-Whitney U test p-value: {p_value:.4f}")
            if p_value < 0.05:
                print(f"  ✓ Significant difference between groups")
            else:
                print(f"  ✗ No significant difference")


BIVARIATE ANALYSIS (Features vs Fraud) - CLEANED DATA

AMOUNT by Fraud Status:
----------------------------------------
           count        mean         std   min    25%     50%      75%      max
is_fraud                                                                       
0         9848.0  175.350818  173.986699  0.01  50.99  122.12  241.660  1471.04
1          151.0  216.182980  248.120467  0.11  41.53  118.94  341.695  1185.07

  Mann-Whitney U test p-value: 0.5734
  ✗ No significant difference

TRANSACTION_HOUR by Fraud Status:
----------------------------------------
           count       mean       std  min  25%   50%   75%   max
is_fraud                                                         
0         9848.0  11.711718  6.871173  0.0  6.0  12.0  18.0  23.0
1          151.0   3.841060  5.803554  0.0  1.0   2.0   3.0  23.0

  Mann-Whitney U test p-value: 0.0000
  ✓ Significant difference between groups

DEVICE_TRUST_SCORE by Fraud Status:
--------------------------------

## 8. Save Cleaned Data

In [48]:
print("\n" + "="*80)
print("💾 SAVING CLEANED DATA")
print("="*80)

# Save cleaned data
df.to_csv(output_filepath, index=False)
print(f"\n✅ Cleaned data saved to: {output_filepath}")
print(f"✅ Total records saved: {len(df):,}")
print(f"✅ Total features: {len(df.columns)}")

# File size
import os
file_size = os.path.getsize(output_filepath) / 1024  # KB
print(f"✅ File size: {file_size:.2f} KB")

print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)
print(f"\n✅ Data Cleaning Completed")
print(f"✅ Exploratory Data Analysis Completed")
print(f"✅ Data Quality Score: {overall_quality:.2f}%")
print(f"✅ Records: {rows_before:,} → {len(df):,} ({len(df)/rows_before*100:.2f}% retained)")
print(f"✅ Ready for Feature Engineering & Modeling")
print("\n" + "="*80)


💾 SAVING CLEANED DATA

✅ Cleaned data saved to: Cleaned_Data/credit_card_fraud_10k_cleaned.csv
✅ Total records saved: 9,999
✅ Total features: 10
✅ File size: 352.04 KB

EXECUTIVE SUMMARY

✅ Data Cleaning Completed
✅ Exploratory Data Analysis Completed
✅ Data Quality Score: 100.00%
✅ Records: 10,000 → 9,999 (99.99% retained)
✅ Ready for Feature Engineering & Modeling



## 9. Data Comparison: Before vs After Cleaning

In [49]:
print("\n" + "="*80)
print("📊 BEFORE vs AFTER CLEANING COMPARISON")
print("="*80)

comparison = pd.DataFrame({
    'Metric': [
        'Total Records',
        'Missing Values',
        'Duplicates',
        'Invalid Amounts',
        'Domain Violations',
        'Data Quality Score'
    ],
    'Before Cleaning': [
        f"{rows_before:,}",
        f"{df_raw.isnull().sum().sum()}",
        f"{df_raw.duplicated().sum()}",
        f"{len(df_raw[df_raw['amount'] <= 0])}",
        f"{total_invalid}",
        "98.33%"  # From original output
    ],
    'After Cleaning': [
        f"{len(df):,}",
        f"{df.isnull().sum().sum()}",
        f"{df.duplicated().sum()}",
        f"{len(df[df['amount'] <= 0])}",
        f"{sum(verification_results.values())}",
        f"{overall_quality:.2f}%"
    ]
})

print("\n" + comparison.to_string(index=False))
print("\n" + "="*80)


📊 BEFORE vs AFTER CLEANING COMPARISON

            Metric Before Cleaning After Cleaning
     Total Records          10,000          9,999
    Missing Values               0              0
        Duplicates               0              0
   Invalid Amounts               1              0
 Domain Violations               0              0
Data Quality Score          98.33%        100.00%



## 10. Quick Data Preview

In [1]:
print("\nLEANED DATA PREVIEW")
print("="*80)
print("\nFirst 10 rows:")
df.head(10)


LEANED DATA PREVIEW

First 10 rows:


NameError: name 'df' is not defined

In [ ]:
print("\nDATA INFO")
print("="*80)
df.info()


📋 DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   transaction_id       9999 non-null   int64  
 1   amount               9999 non-null   float64
 2   transaction_hour     9999 non-null   int64  
 3   merchant_category    9999 non-null   object 
 4   foreign_transaction  9999 non-null   int64  
 5   location_mismatch    9999 non-null   int64  
 6   device_trust_score   9999 non-null   int64  
 7   velocity_last_24h    9999 non-null   int64  
 8   cardholder_age       9999 non-null   int64  
 9   is_fraud             9999 non-null   int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 781.3+ KB


In [ ]:
print("\nSTATISTICAL SUMMARY")
print("="*80)
df.describe()


📋 STATISTICAL SUMMARY


,transaction_id,amount,transaction_hour,foreign_transaction,location_mismatch,device_trust_score,velocity_last_24h,cardholder_age,is_fraud
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000,9999.000000
mean,5000.709771,175.967446,11.592859,0.097810,0.085709,61.797880,2.009001,43.470747,0.015102
std,2886.963832,175.392771,6.922914,0.297072,0.279947,21.487885,1.432595,14.978498,0.121963
min,1.000000,0.010000,0.000000,0.000000,0.000000,25.000000,0.000000,18.000000,0.000000
25%,2500.500000,50.915000,6.000000,0.000000,0.000000,43.000000,1.000000,30.000000,0.000000
50%,5001.000000,122.100000,12.000000,0.000000,0.000000,62.000000,2.000000,44.000000,0.000000
75%,7500.500000,242.550000,18.000000,0.000000,0.000000,80.000000,3.000000,56.000000,0.000000
max,10000.000000,1471.040000,23.000000,1.000000,1.000000,99.000000,9.000000,69.000000,1.000000
